In [ ]:
%matplotlib inline
import sys
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/')
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import fill_values as fv
import calc_station_properties as cs

import createFolder as cF

In [ ]:
def create_netCDF_for_station(thredds,year,month,day,t,stn_lon,stn_lat,ens_memb):
    fn = netCDF4.Dataset('%s/%s/%s/%s/meps_extracted_2_5km_%s%s%sT%sZ.nc' %(thredds,year,month,day,year,month,day,t),'r')

## Latitudes
## [y = 949][x = 739]
    latitude = fn.variables['latitude']

## Longitudes 
## [y = 949][x = 739]
    longitude = fn.variables['longitude']

# Now find the absolute value of the difference between the  station's lat/lon with every point in the grid. 
    x,y = cs.find_station_yx(latitude, longitude, stn_lat, stn_lon)

    time = fn.variables['time']
    height0 = fn.variables['height0']
    
    air_temperature_0m = cs.get_value_at_station(fn, 'air_temperature_0m',ens_memb,x,y)

    graupelfall_amount = cs.get_value_at_station(fn, 'graupelfall_amount',ens_memb,x,y)
    liquid_water_content_of_surface_snow = cs.get_value_at_station(fn, 'liquid_water_content_of_surface_snow',
                                                                       ens_memb,x,y)
    precipitation_amount_acc = cs.get_value_at_station(fn, 'precipitation_amount_acc',ens_memb,x,y)
    rainfall_amount = cs.get_value_at_station(fn, 'rainfall_amount',ens_memb,x,y)
    snowfall_amount = cs.get_value_at_station(fn, 'snowfall_amount',ens_memb,x,y)
    surface_air_pressure = cs.get_value_at_station(fn, 'surface_air_pressure',ens_memb,x,y)
    surface_geopotential = cs.get_value_at_station(fn, 'surface_geopotential',ens_memb,x,y)

    surface = [
                  air_temperature_0m,                      ## surface temperature
                  graupelfall_amount,                      ## graupelfall amount
                  liquid_water_content_of_surface_snow,    ## Snow water equivalent
                  precipitation_amount_acc,                ## accumulated  total precipitation
                  rainfall_amount,                         ## instantanous rainfall at surface
                  snowfall_amount,                         ## instantaneous snowfall amount at surface
                  surface_air_pressure,                     ## ps: surface_air_pressure
                  surface_geopotential                     ## Surface geopotential (fis) 
                       ]
    sfc = [
                  'air_temperature_0m',                      ## surface temperature
                  'graupelfall_amount',                      ## graupelfall amount
                  'liquid_water_content_of_surface_snow',    ## Snow water equivalent
                  'precipitation_amount_acc',                ## accumulated  total precipitation
                  'rainfall_amount',                         ## instantanous rainfall at surface
                  'snowfall_amount',                         ## instantaneous snowfall amount at surface
                  'surface_air_pressure',                    ## ps: surface_air_pressure
                  'surface_geopotential'
               ]

### write netCDF file
    f = netCDF4.Dataset('%s/%s/sfc_%s/%s%s%s_%s_%s.nc' %(dirnc,stn_name,t,year,month,day,t,ens_memb), 'w')

### create dimensions
    f.createDimension('time', time.shape[0])
    f.createDimension('height0', height0.shape[0])

    t = f.createVariable('time', time.dtype,'time',zlib = True)
    h = f.createVariable('height0', height0.dtype, 'height0', zlib=True)
    dim = ('time', 'height0')
#    at_0m = f.createVariable(varname='air_temperature_0m', datatype=air_temperature_0m.dtype, dimensions=dim,
 #                                     fill_value = air_temperature_0m.fill_value, zlib=True)


    t[:] = time[:]
    h[:] = height0[:]
######## with Vertical Levels ( height0 ) #################################

    dim = ('time', 'height0')
    at_0m = cs.get_netCDF_variable(f,'air_temperature_0m', air_temperature_0m,dim)
    ga_0m = cs.get_netCDF_variable(f,'graupelfall_amount', graupelfall_amount,dim)
    lwc_0m = cs.get_netCDF_variable(f,'liquid_water_content_of_surface_snow', liquid_water_content_of_surface_snow,dim)
    pr_0m = cs.get_netCDF_variable(f,'precipitation_amount_acc',precipitation_amount_acc,dim)
    ra_0m = cs.get_netCDF_variable(f,'rainfall_amount',rainfall_amount,dim)
    sa_0m = cs.get_netCDF_variable(f,'snowfall_amount',snowfall_amount,dim)
    ps = cs.get_netCDF_variable(f,'surface_air_pressure',surface_air_pressure,dim)
    geop = cs.get_netCDF_variable(f,'surface_geopotential',surface_geopotential,dim)

    
    f.close()
    fn.close()
 #   ens_memb = '%s' %(ens_memb)
#    print('File written: %s%s%s_%s_%s.nc' %(year, month, day, time, ens_memb))

In [ ]:
year = '2016'
month = '12'
#day = '20'
time = '18'
#time = '00'
stn_lat = 59.8
stn_lon = 7.2
stn_name = 'Haukeliseter'
dirnc = '../../MEPS_data'
cF.createFolder('%s/%s/sfc_%s/' %(dirnc,stn_name,time))
thredds  = 'http://thredds.met.no/thredds/dodsC/meps25epsarchive'

In [ ]:
t = [
    1, 2, 3, 
    4, 5, 6 , 
    7, 8, 9,10, 11,
    12,13,14,15,    
    17,18,19,20, 
    21,22,23,
    24,25,26,
    27,28,29,30, 
    31
    ]

In [ ]:
for day in t:
    if day < 10:
        day = '0%s' %(day)
    else:
        day = '%s' %day
    for ens_memb in range(0,10):
        create_netCDF_for_station(thredds,year,month,day,time,stn_lon,stn_lat,ens_memb)
        print('file written: %s%s%s_%s.nc' %(year,month,day,ens_memb))